<a href="https://colab.research.google.com/github/mr-alamdari/NLP-Text-Generation/blob/main/NLP_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
import numpy as np
from pickle import dump, load

In [2]:
def read_file(path):
  with open(path) as f:
    my_str = f.read()
  return my_str

In [3]:
!wget https://raw.githubusercontent.com/mr-alamdari/NLP-Text-Generation/main/moby_dick_four_chapters.txt

--2022-05-03 11:42:41--  https://raw.githubusercontent.com/mr-alamdari/NLP-Text-Generation/main/moby_dick_four_chapters.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62685 (61K) [text/plain]
Saving to: ‘moby_dick_four_chapters.txt’

moby_dick_four_chap 100%[===================>]  61.22K  --.-KB/s    in 0.001s  

2022-05-03 11:42:41 (66.8 MB/s) - ‘moby_dick_four_chapters.txt’ saved [62685/62685]



In [4]:
mobydic = read_file('moby_dick_four_chapters.txt')

In [5]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [6]:
nlp.max_length = 1198623

In [7]:
seperate_punc = lambda doc: [token.text.lower() for token in nlp(doc) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']

In [8]:
tokens = seperate_punc(mobydic)

In [10]:
tokens[20:40]

['nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on',
 'shore',
 'i',
 'thought',
 'i',
 'would',
 'sail',
 'about',
 'a',
 'little',
 'and',
 'see',
 'the',
 'watery',
 'part']

In [11]:
len(tokens)

11338

In [12]:
train_len = 25 + 1 # 25 training words , then one target word

text_sequences = [tokens[i-train_len: i] for i in range(train_len, len(tokens))]

In [13]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [14]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [15]:
' '.join(text_sequences[2])

'ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i'

In [16]:
' '.join(text_sequences[3])

'some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i thought'

In [17]:
import tensorflow as tf

In [18]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [19]:
tokenizer.fit_on_texts(text_sequences)

In [20]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [21]:
sequences = np.array(sequences)

In [22]:
print(sequences[0])

[ 956   14  263   51  261  408   87  219  129  111  954  260   50   43
   38  315    7   23  546    3  150  259    6 2712   14   24]


In [23]:
print(sequences[1])

[  14  263   51  261  408   87  219  129  111  954  260   50   43   38
  315    7   23  546    3  150  259    6 2712   14   24  957]


In [24]:
print(sequences[2])

[ 263   51  261  408   87  219  129  111  954  260   50   43   38  315
    7   23  546    3  150  259    6 2712   14   24  957    5]


In [25]:
print(sequences[3])

[  51  261  408   87  219  129  111  954  260   50   43   38  315    7
   23  546    3  150  259    6 2712   14   24  957    5   60]


In [26]:
for i in sequences[0]:
  print(i, tokenizer.index_word[i])

956 call
14 me
263 ishmael
51 some
261 years
408 ago
87 never
219 mind
129 how
111 long
954 precisely
260 having
50 little
43 or
38 no
315 money
7 in
23 my
546 purse
3 and
150 nothing
259 particular
6 to
2712 interest
14 me
24 on


In [27]:
for i, count in enumerate(sorted(tokenizer.word_counts.items(), key=lambda x: -x[1])):
  if i == 30:
    break
  print(i, count)

0 ('the', 15540)
1 ('a', 10377)
2 ('and', 9646)
3 ('of', 8287)
4 ('i', 7150)
5 ('to', 6497)
6 ('in', 5647)
7 ('it', 4238)
8 ('that', 3770)
9 ('he', 3247)
10 ('his', 3139)
11 ('was', 2886)
12 ('but', 2652)
13 ('me', 2471)
14 ('with', 2392)
15 ('as', 2366)
16 ('at', 2184)
17 ('this', 2158)
18 ('you', 2158)
19 ('is', 1950)
20 ('all', 1872)
21 ('for', 1820)
22 ('my', 1786)
23 ('on', 1716)
24 ('be', 1716)
25 ("'s", 1691)
26 ('not', 1534)
27 ('from', 1508)
28 ('there', 1456)
29 ('one', 1300)


In [28]:
vocab_size=len(tokenizer.word_counts)

In [29]:
vocab_size

2717

In [30]:
x = sequences[:, :-1]

In [31]:
y = sequences[:, -1]

In [32]:
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size+1)

In [33]:
seq_len = x.shape[1]

In [34]:
def create_model(vocab_size, seq_len):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Embedding(vocab_size, seq_len, input_length=seq_len))
  model.add(tf.keras.layers.LSTM(seq_len, return_sequences=True))
  model.add(tf.keras.layers.LSTM(seq_len))
  model.add(tf.keras.layers.Dense(seq_len, activation='relu'))
  model.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))
  model.compile(loss='categorical_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

  model.summary()
  return model

In [35]:
model = create_model(vocab_size+1, seq_len)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            67950     
                                                                 
 lstm (LSTM)                 (None, 25, 25)            5100      
                                                                 
 lstm_1 (LSTM)               (None, 25)                5100      
                                                                 
 dense (Dense)               (None, 25)                650       
                                                                 
 dense_1 (Dense)             (None, 2718)              70668     
                                                                 
Total params: 149,468
Trainable params: 149,468
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(x, y, batch_size=32, epochs=200, verbose=2)

Epoch 1/200
354/354 - 8s - loss: 6.1576 - accuracy: 0.0517 - 8s/epoch - 23ms/step
Epoch 2/200
354/354 - 8s - loss: 6.0735 - accuracy: 0.0530 - 8s/epoch - 23ms/step
Epoch 3/200
354/354 - 8s - loss: 5.9708 - accuracy: 0.0541 - 8s/epoch - 23ms/step
Epoch 4/200
354/354 - 8s - loss: 5.8935 - accuracy: 0.0595 - 8s/epoch - 23ms/step
Epoch 5/200
354/354 - 8s - loss: 5.8123 - accuracy: 0.0656 - 8s/epoch - 23ms/step
Epoch 6/200
354/354 - 8s - loss: 5.7355 - accuracy: 0.0663 - 8s/epoch - 24ms/step
Epoch 7/200
354/354 - 8s - loss: 5.6799 - accuracy: 0.0681 - 8s/epoch - 23ms/step
Epoch 8/200
354/354 - 8s - loss: 5.6341 - accuracy: 0.0667 - 8s/epoch - 23ms/step
Epoch 9/200
354/354 - 8s - loss: 5.5914 - accuracy: 0.0674 - 8s/epoch - 23ms/step
Epoch 10/200
354/354 - 8s - loss: 5.5460 - accuracy: 0.0682 - 8s/epoch - 23ms/step
Epoch 11/200
354/354 - 8s - loss: 5.5051 - accuracy: 0.0703 - 8s/epoch - 23ms/step
Epoch 12/200
354/354 - 8s - loss: 5.4649 - accuracy: 0.0714 - 8s/epoch - 23ms/step
Epoch 13/200


In [39]:
model.save('mobydic_model.h5')

In [40]:
dump(tokenizer, open('simpleTokenizer', 'wb'))

In [41]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
  output_text = []
  input_text = seed_text
  for i in range(num_gen_words):
    encoded_text = tokenizer.texts_to_sequences([input_text])[0]
    pad_encoded = tf.keras.preprocessing.sequence.pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
    pred_word = model.predict(pad_encoded, verbose=0)[0]
    pred_word_ind = np.argmax(pred_word)
    pred_word = tokenizer.index_word[pred_word_ind]
    input_text += ' '+pred_word
    output_text.append(pred_word)
  return ' '.join(output_text)

In [42]:
num_gen_words = 20
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

'aggregated years stand war than ye this harpooneer might be and exactly try the piece of knowing boots of him'

In [43]:
model.fit(x, y, batch_size=32, epochs=100, verbose=2)

Epoch 1/100
354/354 - 8s - loss: 1.3031 - accuracy: 0.6761 - 8s/epoch - 23ms/step
Epoch 2/100
354/354 - 8s - loss: 1.3128 - accuracy: 0.6713 - 8s/epoch - 23ms/step
Epoch 3/100
354/354 - 8s - loss: 1.2964 - accuracy: 0.6795 - 8s/epoch - 23ms/step
Epoch 4/100
354/354 - 8s - loss: 1.2651 - accuracy: 0.6858 - 8s/epoch - 23ms/step
Epoch 5/100
354/354 - 8s - loss: 1.2750 - accuracy: 0.6818 - 8s/epoch - 23ms/step
Epoch 6/100
354/354 - 8s - loss: 1.2462 - accuracy: 0.6910 - 8s/epoch - 23ms/step
Epoch 7/100
354/354 - 8s - loss: 1.2508 - accuracy: 0.6844 - 8s/epoch - 23ms/step
Epoch 8/100
354/354 - 8s - loss: 1.2353 - accuracy: 0.6917 - 8s/epoch - 23ms/step
Epoch 9/100
354/354 - 8s - loss: 1.2318 - accuracy: 0.6932 - 8s/epoch - 23ms/step
Epoch 10/100
354/354 - 8s - loss: 1.2352 - accuracy: 0.6933 - 8s/epoch - 23ms/step
Epoch 11/100
354/354 - 8s - loss: 1.2122 - accuracy: 0.7016 - 8s/epoch - 23ms/step
Epoch 12/100
354/354 - 8s - loss: 1.2036 - accuracy: 0.6993 - 8s/epoch - 23ms/step
Epoch 13/100


In [44]:
num_gen_words = 40
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

'i never unaccountable hours and had not help the decks in anxious sort of fooling it from what i told it for he continued the captain and to the answer i am two and nine at my bit of short'

In [45]:
model.fit(x, y, batch_size=32, epochs=100, verbose=2)

Epoch 1/100
354/354 - 8s - loss: 0.7558 - accuracy: 0.8128 - 8s/epoch - 23ms/step
Epoch 2/100
354/354 - 8s - loss: 0.7329 - accuracy: 0.8169 - 8s/epoch - 23ms/step
Epoch 3/100
354/354 - 8s - loss: 0.7533 - accuracy: 0.8122 - 8s/epoch - 23ms/step
Epoch 4/100
354/354 - 8s - loss: 0.7293 - accuracy: 0.8211 - 8s/epoch - 24ms/step
Epoch 5/100
354/354 - 8s - loss: 0.7326 - accuracy: 0.8192 - 8s/epoch - 23ms/step
Epoch 6/100
354/354 - 8s - loss: 0.7416 - accuracy: 0.8119 - 8s/epoch - 23ms/step
Epoch 7/100
354/354 - 8s - loss: 0.7505 - accuracy: 0.8120 - 8s/epoch - 23ms/step
Epoch 8/100
354/354 - 8s - loss: 0.7325 - accuracy: 0.8221 - 8s/epoch - 23ms/step
Epoch 9/100
354/354 - 8s - loss: 0.7131 - accuracy: 0.8230 - 8s/epoch - 23ms/step
Epoch 10/100
354/354 - 8s - loss: 0.7331 - accuracy: 0.8134 - 8s/epoch - 23ms/step
Epoch 11/100
354/354 - 8s - loss: 0.7835 - accuracy: 0.8002 - 8s/epoch - 23ms/step
Epoch 12/100
354/354 - 8s - loss: 0.7418 - accuracy: 0.8075 - 8s/epoch - 23ms/step
Epoch 13/100


In [46]:
model.save('mobydic_model.h5')
num_gen_words = 40
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

"of spending a sufferable night unless in some other person 's looking but the bench i think the piece of life for hands glided out to those place in him and if it 's turned over upon a grunt and"

In [47]:
model.fit(x, y, batch_size=32, epochs=100, verbose=2)

Epoch 1/100
354/354 - 8s - loss: 0.4524 - accuracy: 0.8850 - 8s/epoch - 23ms/step
Epoch 2/100
354/354 - 8s - loss: 0.4721 - accuracy: 0.8800 - 8s/epoch - 23ms/step
Epoch 3/100
354/354 - 8s - loss: 0.4530 - accuracy: 0.8873 - 8s/epoch - 23ms/step
Epoch 4/100
354/354 - 8s - loss: 0.4611 - accuracy: 0.8860 - 8s/epoch - 24ms/step
Epoch 5/100
354/354 - 8s - loss: 0.4741 - accuracy: 0.8807 - 8s/epoch - 23ms/step
Epoch 6/100
354/354 - 8s - loss: 0.4427 - accuracy: 0.8910 - 8s/epoch - 23ms/step
Epoch 7/100
354/354 - 8s - loss: 0.4480 - accuracy: 0.8879 - 8s/epoch - 23ms/step
Epoch 8/100
354/354 - 8s - loss: 0.4983 - accuracy: 0.8723 - 8s/epoch - 23ms/step
Epoch 9/100
354/354 - 8s - loss: 0.4736 - accuracy: 0.8794 - 8s/epoch - 23ms/step
Epoch 10/100
354/354 - 8s - loss: 0.4286 - accuracy: 0.8919 - 8s/epoch - 23ms/step
Epoch 11/100
354/354 - 8s - loss: 0.4434 - accuracy: 0.8937 - 8s/epoch - 23ms/step
Epoch 12/100
354/354 - 8s - loss: 0.4171 - accuracy: 0.8984 - 8s/epoch - 23ms/step
Epoch 13/100


In [48]:
model.save('mobydic_model.h5')
num_gen_words = 80
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

'and that stranger a harpooneer then your objections indefinitely multiply nor was there any earthly reason why i as a sailor should sleep two in a bed at last can instinct slept goes the answer what a neck however my own wooden house there shelf what this shabby teeth house where so wildly other in the overwhelming idea of a tomahawk fish?"--this almost cases besmoked in all this harpoon air into me into the vengeance voyage of the legs morning'

In [49]:
model.fit(x, y, batch_size=32, epochs=100, verbose=2)

Epoch 1/100
354/354 - 8s - loss: 0.4217 - accuracy: 0.9006 - 8s/epoch - 23ms/step
Epoch 2/100
354/354 - 8s - loss: 0.3487 - accuracy: 0.9186 - 8s/epoch - 23ms/step
Epoch 3/100
354/354 - 8s - loss: 0.2617 - accuracy: 0.9431 - 8s/epoch - 23ms/step
Epoch 4/100
354/354 - 8s - loss: 0.2268 - accuracy: 0.9532 - 8s/epoch - 23ms/step
Epoch 5/100
354/354 - 8s - loss: 0.2247 - accuracy: 0.9516 - 8s/epoch - 23ms/step
Epoch 6/100
354/354 - 8s - loss: 0.2509 - accuracy: 0.9456 - 8s/epoch - 23ms/step
Epoch 7/100
354/354 - 8s - loss: 0.3435 - accuracy: 0.9135 - 8s/epoch - 23ms/step
Epoch 8/100
354/354 - 8s - loss: 0.3931 - accuracy: 0.8981 - 8s/epoch - 23ms/step
Epoch 9/100
354/354 - 8s - loss: 0.3548 - accuracy: 0.9055 - 8s/epoch - 23ms/step
Epoch 10/100
354/354 - 8s - loss: 0.3207 - accuracy: 0.9189 - 8s/epoch - 23ms/step
Epoch 11/100
354/354 - 8s - loss: 0.2797 - accuracy: 0.9331 - 8s/epoch - 23ms/step
Epoch 12/100
354/354 - 8s - loss: 0.2673 - accuracy: 0.9393 - 8s/epoch - 23ms/step
Epoch 13/100


In [50]:
model.save('mobydic_model.h5')
num_gen_words = 80
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

"out of the bed and not dreadful the i the the ibis and perhaps queequeg river hand in his uncomfortableness with water his hacking horrifying implement mixed with an poison one meridians chimney leaves at those one on i begged any green deal as there that any sober bedfellow and hand and one there sam i thought paid the middle of the streets received as it was not seem such want once complied in unlacing being found it 's has"

In [51]:
model.fit(x, y, batch_size=32, epochs=100, verbose=2)

Epoch 1/100
354/354 - 8s - loss: 0.1674 - accuracy: 0.9638 - 8s/epoch - 24ms/step
Epoch 2/100
354/354 - 8s - loss: 0.2275 - accuracy: 0.9449 - 8s/epoch - 24ms/step
Epoch 3/100
354/354 - 8s - loss: 0.3381 - accuracy: 0.9119 - 8s/epoch - 23ms/step
Epoch 4/100
354/354 - 8s - loss: 0.3276 - accuracy: 0.9135 - 8s/epoch - 24ms/step
Epoch 5/100
354/354 - 8s - loss: 0.2743 - accuracy: 0.9342 - 8s/epoch - 24ms/step
Epoch 6/100
354/354 - 8s - loss: 0.1985 - accuracy: 0.9558 - 8s/epoch - 24ms/step
Epoch 7/100
354/354 - 8s - loss: 0.1529 - accuracy: 0.9702 - 8s/epoch - 23ms/step
Epoch 8/100
354/354 - 8s - loss: 0.1497 - accuracy: 0.9723 - 8s/epoch - 23ms/step
Epoch 9/100
354/354 - 8s - loss: 0.1406 - accuracy: 0.9759 - 8s/epoch - 24ms/step
Epoch 10/100
354/354 - 8s - loss: 0.1569 - accuracy: 0.9680 - 8s/epoch - 24ms/step
Epoch 11/100
354/354 - 8s - loss: 0.2597 - accuracy: 0.9316 - 8s/epoch - 24ms/step
Epoch 12/100
354/354 - 8s - loss: 0.4263 - accuracy: 0.8889 - 8s/epoch - 24ms/step
Epoch 13/100


In [52]:
model.save('mobydic_model.h5')
num_gen_words = 80
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

'leave me to dress afterwards leaving the whole apartment to myself thinks i queequeg under the circumstances this is a very civilized overture but the truth is these savages have an innate sense of delicacy say what you will it is marvellous how essentially polite they are i pay this particular compliment to queequeg because he treated me with so much civility and consideration while i was guilty of great rudeness staring at him from the bed and watching all'

In [53]:
num_gen_words = 80

test_text = 'Here I thought, every thing wa finished'

generate_text(model, tokenizer, seq_len, test_text, num_gen_words)

"the bowsprit reg'lar about the bag apartment it down as the bag and then smoking such two however and were there too ai no green stage trials on its sighs did so good fire and got a cannibal off from a bed like a bit and man stuck over to that sail of a cataract of how seem but cunningly somewhere ashore to be broad idea of sleeping from it sabbee of it were checkered with his old persians going"

In [54]:
num_gen_words = 80

test_text = 'I am very happy about'

generate_text(model, tokenizer, seq_len, test_text, num_gen_words)

"together arrived at a hamper ' tell 's all the great stove and the bench vigorously before a difference too not had not not to have unite come to please into a room sleeping upon great monkey jackets of scalding before the whaling into my creatures of icicles abominable followed the horse collar and at the quilt his wrist in me with his bed is when it must have go into the room or taking care of sailors no adjoining"